# 🏢 Azure AI Foundry 에이전트

<div style="background: linear-gradient(135deg, #0078d4 0%, #00bcf2 100%); padding: 10px; border-radius: 7px; color: white; margin-bottom: 20px;">

### ☁️ 클라우드 기반 에이전트 서비스

Microsoft Agent Framework(MAF)를 사용하여 Azure AI Foundry Agent 서비스 기반의 강력한 에이전트를 구축하세요.

</div>

**✨ 주요 기능**

| 기능 | 설명 |
|------|------|
| 🗨️ **관리형 대화 스레드** | 서비스에서 자동으로 대화 컨텍스트 관리 |
| 💾 **영구 저장소** | 에이전트 상태가 클라우드에 안전하게 보관 |
| 📈 **확장성** | 자동 스케일링으로 대규모 워크로드 처리 |
| 🔒 **보안** | Azure 보안 정책 및 규정 준수 기본 제공 |

---

## 1️⃣ 빠른 시작: 기본 에이전트 생성

<div style="background: #e3f2fd; border-left: 4px solid #2196f3; padding: 15px; border-radius: 8px; margin: 15px 0;">

### ⚡ AzureAIAgentClient로 즉시 시작

가장 빠르고 간단하게 Azure AI Foundry 에이전트를 생성하는 방법입니다.  
AzureAIAgentClient로 생성된 에이전트는 Foundry Portal(Classic)에 등록되고 보여집니다.

**주요 특징:**
- 🚀 **빠른 시작**: 몇 줄의 코드로 에이전트 생성
- 🧪 **테스트 용도**: 프로토타이핑과 실험에 최적화

</div>

In [ ]:
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential
from agent_framework import Agent

agent = Agent(
    client=AzureAIAgentClient(
        credential=DefaultAzureCredential(),
        agent_name="KoreanAgent"
    ),
    name="Assistant",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨",
)

result = await agent.run("안녕하세요!")
print(result.text)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000027032EF81D0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000027032E3D9A0>


안녕하세요! 😊 두 번이나 인사해 주셔서 너무 반가운데요. 어떻게 도와드릴까요?


상기 코드를 실행한 뒤, Foundry Portal(Classic)에 가보면 "KoreanAgent"라는 이름의 에이전트가 생성된 것을 확인할 수 있습니다.

![Foundry Portal에서 생성된 에이전트](./images/aiagent-portal.png)

---

## 2️⃣ 에이전트 생성 및 관리 (클래식 포털)

또는, Foundry SDK에서 제공하는 AgentsClient를 사용해서, 에이전트가 아직 존재하지 않을 경우에만 새로 생성하고, 이미 존재한다면 기존 에이전트를 불러와 사용할 수도 있습니다.

> 📌 **포털 위치**: 이 방식으로 생성된 에이전트는 **클래식 포털**에 표시됩니다.

In [ ]:
import os
from agent_framework.azure import AzureAIAgentClient
from agent_framework import Agent
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient

# 이 Agent는 신규 Foundry 포탈이 아닌, Classic 포탈에 생성됩니다.
credential = DefaultAzureCredential()
agents_client = AgentsClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential
)

#새로 생성할 에이전트 이름
agent_name="PersistentAgent"

# 기존 에이전트 목록 조회
print("=== 기존 에이전트 목록 ===")
for agent in agents_client.list_agents():
    print(f"Agent ID: {agent.id}, Name: {agent.name}")

# 기존 Agent가 없으면 새로운 Agent 생성
agent = None
for existing in agents_client.list_agents():
    if existing.name == agent_name:
        agent = existing
        break

if(agent):
    print(f"에이전트 '{agent_name}'(이)가 이미 존재합니다. 새로 생성하지 않습니다.")
else:
    print(f"에이전트 '{agent_name}'(이)가 존재하지 않습니다. 새로 생성합니다.")
    agent = agents_client.create_agent(
        model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
        name=agent_name,
        instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨"
    )

print(f"에이전트 ID: {agent.id}")
print(f"에이전트 이름: {agent.name}")
print(f"모델: {agent.model}")
print(f"instruction : {agent.instructions}")

# 기존(또는 생성된) 에이전트를 사용하여 ChatAgent 구성
chat_agent = Agent(
    client=AzureAIAgentClient(
        credential=credential,
        agent_id=agent.id
    ),
    name="Assistant",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨",
    store=True
)
result = await chat_agent.run("오늘도 좋은 하루!")
print(result.text)

agent_id=agent.id
# 정리 (필요시 주석 해제)
# agents_client.delete_agent(agent.id)
# print(f"\n에이전트 삭제 완료: {agent.id}")

---

## 3️⃣ 기존 에이전트 활용하기 (클래식 포털)

<div style="background: #e8f5e9; border-left: 4px solid #4caf50; padding: 10px; border-radius: 8px; margin: 10px 0;">

### 🔗 에이전트 ID로 즉시 연결

이미 생성된 에이전트를 재사용하여 빠르게 시작할 수 있습니다.

</div>

**💡 활용 시나리오:**

- ♻️ **재사용**: 이전에 생성한 에이전트 다시 활용
- 👥 **협업**: 팀원이 만든 공유 에이전트 접근
- 🏭 **프로덕션**: 검증된 안정적인 에이전트 연결
- 🎯 **일관성**: 동일한 설정과 동작 보장

In [ ]:
from agent_framework import Agent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

agent_id="<이전의 agent id>"  # 이전에 생성된 에이전트를 여기에 할당
agent_id="asst_iV9tCjf2fjVFo7yNdN7PWZ60" # 테스트용 하드코딩, 실제로는 안전하게 관리 필요

agent = Agent(
    client=AzureAIAgentClient(
        credential=credential,
        agent_id=agent_id
    ),
    name="Assistant",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨",
    store=True
)
        
result = await agent.run("한국은 어디에 있나요?")
print(result.text)


---

## 🚀 지원되는 고급 기능

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 12px; color: white; margin: 15px 0;">

Azure AI Foundry 에이전트는 엔터프라이즈급 고급 기능을 기본 제공합니다.

</div>

**핵심 기능:**

| 기능 | 설명 |
|------|------|
|🔧 **Function Tools** | 사용자 정의 함수 및 외부 도구 통합 |
| 💻 **Code Interpreter** | Python 코드 실시간 실행 및 데이터 분석 | 
| 📡 **Streaming Responses** | 점진적 응답으로 향상된 UX 제공 |
| 📁 **File Handling** | 파일 업로드, 분석 및 생성 |
| 💬 **Multi-turn Conversations** | 컨텍스트 유지 대화 |

---

## 4️⃣ 영구 에이전트 생성 및 관리 (신규 포털)

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 10px; border-radius: 7px; color: white; margin: 5px 0;">

### 🆕 AIProjectClient로 차세대 에이전트 구축

**참고**:

현재는 MAF의 종속성으로 제공되는 에이전트는 모두 클래식 포털에서 관리되고 있지만,   
향후 AIProjectClient를 통해 신규 포털에서도 영구 에이전트를 생성하고 관리할 수 있게 될 예정입니다.

</div>

**✨ 신규 포털의 강력한 기능:**

| 기능 | 설명 | 이점 |
|------|------|------|
| 📊 **Observability** | 실행 추적 및 성능 모니터링 | 완벽한 가시성 |
| 🔍 **스마트 관리** | 에이전트 자동 감지 및 재사용 | 중복 방지 |
| 🎨 **향상된 UI** | 직관적인 인터페이스 | 쉬운 관리 |
| 🐛 **디버깅 도구** | 상세한 로그 및 트레이싱 | 빠른 문제 해결 |
| 📈 **분석 대시보드** | 에이전트 사용 통계 | 인사이트 확보 |

<div style="background: #fff3cd; border: 1px solid #ffc107; border-radius: 8px; padding: 12px; margin: 15px 0;">

⚠️ **Preview**: 이 기능은 현재 미리 보기 단계입니다. 프로덕션 환경에서 사용 시 주의하세요.

</div>

> 📍 **포털 위치**: 이 방식으로 생성된 에이전트는 **신규 포털**에 표시되며, 모든 최신 기능을 활용할 수 있습니다.

In [ ]:
import os
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import AgentReference, PromptAgentDefinition
from azure.identity import DefaultAzureCredential

# 새로 생성할 에이전트 이름
AGENT_NAME = "BeerAgent"

async with (
    AIProjectClient(
        endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
        credential=DefaultAzureCredential()
    ) as project_client,
):
    # 기존 Agent 찾기
    agent = None
    async for existing in project_client.agents.list():
        if existing.name == AGENT_NAME:
            agent = existing
            break
    
    ## 기존 에이전트가 없으면 새로운 에이전트 생성
    if(agent):
        print(f"에이전트 '{AGENT_NAME}'(이)가 이미 존재합니다. 새로 생성하지 않습니다.")
    else:
        # 에이전트 생성
        print(f"에이전트 '{AGENT_NAME}'(이)가 존재하지 않습니다. 새로 생성합니다.")
        definition = PromptAgentDefinition(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            instructions="당신은 한국어로 답변하는, 맥주와 에일에 정통한 어시스턴트입니다. 🍺✨"
        )                
        agent = await project_client.agents.create(
            name=AGENT_NAME,
            definition=definition
        )
        # 생성된 에이전트 정보 출력
        print(f"생성된 에이전트 이름 : {agent.name}")  

    # OpenAI client 가져오기
    openai_client = project_client.get_openai_client()
    
    # Agent 정보를 본문에 추가하여 Response 생성 요청
    response = await openai_client.responses.create(
        input=[{"role": "user", "content": "겨울에 어울리는 맥주는?"}],
        extra_body={"agent": AgentReference(name=agent.name).as_dict()}
    )
    
    print(f"Agent response: {response.output_text}")

---

## 📸 신규 포털에서 확인하기

<div style="background: #f5f5f5; border-radius: 8px; padding: 20px; margin: 15px 0;">

신규 Azure AI Foundry 포털에서 생성된 에이전트는 다음과 같이 표시됩니다:

![MAF-agent-newportal.png](./images/MAF-agent-newportal.png)

</div>

**🎯 다음 단계:**
- 에이전트에 Function Tools 추가하기
- 멀티턴 대화 구현하기
- 에이전트 성능 모니터링 및 최적화하기